# 002 Linear Regression with Gradient Descent

Let's try to do linear regression in yet another way.

## Predicting Student's grades from Study Hours

Let's predict how well students do based on how much they study. Hey, there should be something there, right??

We take data from an online source, which will give us 100 data points, which we then split into X and Y data.

In [ ]:
%matplotlib inline  
import numpy as np
import matplotlib.pyplot as plt
import sys
import time
from IPython import display

In [ ]:
data = np.genfromtxt('https://raw.githubusercontent.com/kaustubholpadkar/Linear_Regression-Gradient_Descent-Octave/master/data.csv', delimiter=',')
print(data.shape)
X = data[:, 0]
Y = data[:, 1]

What does the data look like?

In [ ]:
fig = plt.figure(figsize=(15,10))
plt.scatter(X, Y)
plt.title('Student Data : Expected Grades vs Hours of Study', fontsize=18)
plt.xlabel('study time [hours]', fontsize=14)
plt.ylabel('expected grade [points]', fontsize=14)
plt.grid()
plt.show()

So, there seems to be some sort of trend here ^^

### Model function

We would like to fit a line to our data, so let's first define the model of a line, given its two parameters (slope $m$ and intercept $b$):

$y = mX+b$

In [ ]:
def line(m, b, X):
    return m * X + b

### Loss function

Next, we need to define our loss function that we want to optimize. As usual, we take the L2, Least-Squares-Loss:

$E(m,b) = \sum_i\left ( Y_i-\text{line}(m,b,X_i) \right )^2$ 

In [ ]:
def loss (m, b, X, Y) :
    return ...

The loss function is a function of two parameters and describes a surface: $E(m,b): \mathrm{R}^2\rightarrow\mathrm{R}$

Let's plot this for a range of parameters:

In [ ]:
ms = np.linspace(-20,20,50)
bs = np.linspace(-40,80,50)

[msm,bsm] = ...

lossSurface = np.zeros((50,50)) 

for i,m in enumerate(ms):
    for j,b in enumerate(bs):
        lossSurface[i,j]=loss(m,b,X,Y)
        
from mpl_toolkits import mplot3d
%matplotlib notebook

fig = plt.figure()
ax = plt.axes(projection='3d')
ax.plot_surface(msm, bsm, lossSurface,cmap='viridis', edgecolor='none')
ax.set_title('Surface plot')
plt.show()

As we can see there seems to be a valley - and hopefully in this valley, we can find a minimum point, which is the lowest point of the loss function and our global optimum!!

Q: How do we find this??

A: We start at a point and walk ...

Q: In which direction do we walk??

A: In the direction that has the ...!

Q: How do we find this direction??

A: It's the ... (two-dimensional!!)

Q: How far do we walk down in this direction

A: ... and may miss our optimum!

### Calculate Gradients

Now, we have to do gradient descent. We will walk in the direction of the negative gradient of the loss function $E$.

The loss function has two parameters $E(m,b)$:

$\frac{\partial E}{\partial m}=-2*\sum (Y-\text{line})*X$

and

$\frac{\partial E}{\partial b}=-2*\sum (Y-\text{line})*(1)$


In [ ]:
# this is the total gradient
def gradient (m, b, X, Y) :
    dm = ...
    db = ...
    return (dm, db)

### Gradient Descent

Now, the very simple gradient descent. In each step, we simply go a little bit in the direction of the negative gradient:

$m_{t+1}=m_t-\text{lr}\frac{\partial E}{\partial m}$

$b_{t+1}=b_t-\text{lr}\frac{\partial E}{\partial b}$

In [ ]:
def gradient_descent (m, b, X, Y, learning_rate, max_iteration,waittime=0.02,onlineplot=True,lossplot=True) :
    errors = np.zeros(max_iteration)
    x = np.arange(25, 75)
    for i in range(max_iteration):
        # calculate the gradients
        dm, db = gradient (m, b, X, Y)
        # update the parameters with a little gradient
        m -= learning_rate * dm
        b -= learning_rate * db
        # evaluate the current line prediction
        y = [m * x_i + b for x_i in x]
        errors[i]=loss(m, b, X, Y)
        print(errors[i])
        # plot it into plot
        if onlineplot:
            plt.figure(figsize=(10,6))
            display.clear_output(wait=True)
            plt.xlim(20, 80)
            plt.ylim(0, 140)
            plt.scatter(X,Y,c='b')
            plt.plot(x,y,c='k')
            plt.grid()
            plt.title('current iteration {}, total loss = {:.2f}'.format(i+1,loss(m,b,X,Y)))
            time.sleep(waittime)
            plt.show()
    # plot the total loss over iterations
    if lossplot:
        fig,ax = plt.subplots(figsize=(8,6))
        plt.plot(errors,'b.-')
        plt.xlabel('Iteration')
        plt.ylabel('Summed Squared Error')
        plt.grid()
    return (m, b)

### Train Model

We have to initialize gradient descent in order to start. Let's start at $m=b=0$, which is a flat line. This is obviously going to be a bad model with a high loss:

In [ ]:
%matplotlib inline
m = 0
b = 0

# print out current total loss and loss per point 
print(loss(m,b,X,Y),loss(m,b,X,Y)/len(Y))

Let's first look at the evolution of the optimization slowly.

In [ ]:
learning_rate = 1e-7
max_iteration = 20
waittime = 1

m = 0
b = 0
m, b = gradient_descent (m, b, X, Y, learning_rate, max_iteration, waittime=waittime)

We can see that the total error goes down nicely across iterations.

Just to see the effect of the learning rate, let's try to learn "faster", by making bigger steps:

In [ ]:
learning_rate = 1e-5
max_iteration = 10

m = 0
b = 0
m, b = gradient_descent (m, b, X, Y, learning_rate, max_iteration, waittime=waittime)

And, that led the optimization to go crazy.

**Remember: gradient descent needs adequate step sizes and hence and adequate learning rate!!!**

In [ ]:
learning_rate = 1e-7
max_iteration = 500

m = 0
b = 0
m, b = gradient_descent (m, b, X, Y, learning_rate, max_iteration, onlineplot=False)

Let's plot the resulting fit together with numpy's ```polyfit``` function result:

In [ ]:
fig = plt.figure(figsize=(10,8))

# Generate data for plotting line after gradient descent
x = np.arange(25, 75)
y = [m * x_i + b for x_i in x]

# numpy linear regression
np_m, np_b = np.polyfit(X, Y, 1)
# Generate data for plotting line from numpy linear regression
np_x = np.arange(25, 75)
np_y = [np_m * x_i + np_b for x_i in np_x]

# plot data and best-fit lines
plt.plot(X, Y, 'o', x, y, '-', np_x, np_y, '--')
plt.legend(loc='upper left', labels=['data points', 'best-fit line', 'numpy line'])

fig.suptitle('Student Data : Expected Grades vs Hours of Study', fontsize=18)
plt.xlabel('study time [hours]', fontsize=14)
plt.ylabel('expected grade [points]', fontsize=14)
plt.grid()
plt.show()

And that's nice!

# Non-linear optimization with Gradient Descent
Obviously, fitting a line is not really the fanciest of applications, since it is well-known that this problem has an analytic solution.

What about non-linear data? Let's first turn to the simple problem of one-dimensional optimization.

We have a function $f(x)$, for which we are trying to find an optimal point, i.e., a point $x_{opt}$, at which $f(x_{opt})=f_{opt}$ is either a minimum or a maximum. 

If we know the gradient then this of course means that $\frac{d}{dx}f(x) = g(x)$ at this point needs to be 0: $g(x_{opt})=0$. 

So, we simply choose a starting point and walk along the function in the direction of the gradient.

In [ ]:
# define the total "loss" function to optimize
def loss_nl(x):
    return(np.cos(x+0.5)*np.exp(0.5*x))

In [ ]:
# gradient of the function
def gradient_nl(x):
    return(-np.sin(x+0.5)*np.exp(0.5*x)+0.5*np.cos(x+0.5)*np.exp(0.5*x))

In [ ]:
# our usual gradient descent algorithm
def gradient_descent_nl(xmax, learning_rate, max_iteration,waittime=0.02,onlineplot=True,lossplot=True) :
    errors = np.zeros(max_iteration)
    x = np.arange(-5,10,0.1)
    for i in range(max_iteration):
        dxmax = gradient_nl(xmax)
        xmax -= learning_rate * dxmax
        y = loss_nl(x)
        errors[i]=dxmax
        if onlineplot:
            plt.figure(figsize=(10,6))
            display.clear_output(wait=True)
            plt.xlim(-5, 10)
            plt.ylim(-200, 200)
            plt.plot(x,y,c='k')
            plt.plot([xmax,xmax],[-200,200],c='r')
            plt.grid()
            plt.title('current iteration {}, optimal point at = {:.2f}'.format(i+1,xmax))
            time.sleep(waittime)
            plt.show()
    if lossplot:
        fig,ax = plt.subplots(figsize=(8,6))
        plt.plot(errors,'b.-')
        plt.xlabel('Iteration')
        plt.ylabel('Summed Squared Error')
        plt.grid()
    return (xmax)

In [ ]:
learning_rate = 1e-1
max_iteration = 30
waittime = 0.1
xstart = 1.01
xmax = gradient_descent_nl (xstart, learning_rate, max_iteration, waittime)

In [ ]:
# more complex function
def loss_nl(x):
    return(np.cos(x+0.5)*np.exp(0.5*np.sin(x)+np.log(x**2)))

# fun derivative
def gradient_nl(x):
    return(np.cos(x + 1/2)*np.exp(np.log(x**2) + np.sin(x)/2)*(np.cos(x)/2 + 2/x) - np.sin(x + 1/2)*np.exp(np.log(x**2) + np.sin(x)/2))

In [ ]:
learning_rate = 1e-2
max_iteration = 30
waittime = 0.1
xstart = -4.9
xmax = gradient_descent_nl (xstart, learning_rate, max_iteration, waittime)

## Fitting to an exponential function with least squares and gradient descent

So, let's apply this idea to another dataset. Here, we take data from the COVID outbreak in the US, available at...

I cut out the 20 days of the initial data, when the death toll due to COVID started to rise.

In [ ]:
data = np.genfromtxt('national-history.csv', delimiter=',')
print(data.shape)
X = np.arange(20)
Y = data[340:360, 2]
Y = np.flip(Y)
fig = plt.figure(figsize=(10,5))
plt.plot(X,Y,'ko-')
plt.grid()
plt.xlabel('Day')
plt.ylabel('Number of COVID-related deaths in the US')
plt.show()


In [ ]:
# define the exponential model
def nl_exp(l, b, X):
    return(np.exp(l*X+b))

# define the least squares loss
def loss_nl (l, b, X, Y) :
    return np.sum(np.square(Y - nl_exp(l, b, X)))

# derivatives of the loss function with regard to the two parameters
def gradient_nl (l, b, X, Y) :
    dl = - 2*np.sum((Y - nl_exp(l, b, X)) * X * np.exp(l*X+b))
    db = - 2*np.sum((Y - nl_exp(l, b, X)) * np.exp(l*X+b))
    return (dl, db)

# actual gradient descent [identical in its logic to the function above]
def gradient_descent_nl (l, b, X, Y, learning_rate, max_iteration,waittime=0.02,onlineplot=True,lossplot=True) :
    errors = np.zeros(max_iteration)
    x = np.arange(0, 20)
    for i in range(max_iteration):
        
        dl, db = gradient_nl (l, b, X, Y)
        l -= learning_rate * dl
        b -= learning_rate * db
        y = [np.exp(l* x_i + b) for x_i in x]
        errors[i]=loss(l, b, X, Y)
        if onlineplot:
            plt.figure(figsize=(10,6))
            display.clear_output(wait=True)
            plt.xlim(0, 20)
            plt.ylim(0, 1200)
            plt.scatter(X,Y,c='b')
            plt.plot(x,y,c='k')
            plt.grid()
            plt.title('current iteration {}, total loss = {:.2f}, l={}, b={}'.format(i+1,loss(m,b,X,Y),l,b))
            time.sleep(waittime)
            plt.show()
    if lossplot:
        fig,ax = plt.subplots(figsize=(8,6))
        plt.plot(errors,'b.-')
        plt.xlabel('Iteration')
        plt.ylabel('Summed Squared Error')
        plt.grid()
    return (m, b)

In [ ]:
print(loss_nl(0.1,1,X,Y))

This is an **extremely** fiddly optimization, as the learning rate can easily overshoot in the later stages. You are welcome to try different starting points and learning rates for this.

In [ ]:
learning_rate = 1e-10
max_iteration = 1000
waittime = 0.01
l = 0.1
b = 1
l, b = gradient_descent_nl (l, b, X, Y, learning_rate, max_iteration, waittime)

So, the question now of course is: is this really an exponential growth? Given the processes of virus transmission, we may have reason to believe so, but as you can see, the overall growth seems sub-exponential.

In addition, we have tried to optimize this with the "raw" exponential function, which - as you can see from the fiddling with learning rates - is really not very convenient for optimization. 

So, there are many better ways to do this -> see your homework assignment.